In [1]:
%run -i ../python/common.py
publish=False

if not publish:
    # cleanup any old state
    # demke - fill in as we see what state gets generated in this page.
    bashCmds('''[[ -d mydir ]] && rm -rf mydir
    #''')
else:
    bashCmds('''rm -rf ~/*''')
    
closeAllOpenTtySessions()



In [2]:
appdir=os.getenv('HOME')
appdir=appdir + "/sync"
output = runTermCmd("[[ -d " + appdir + " ]] &&  rm -rf "+ appdir + 
             ";cp -r ../src/sync " + appdir)

bash = BashSession(cwd=appdir)

In [3]:
# demke:
# 
# This cell is removed in the html, but displays the code in the Jupyter notebook.
#

display(Markdown('<font size="1.2rem">' + FileCodeBox(
    file=appdir + "/counter_mutex.c", 
    lang="", 
    number=True,
    title="<b>counter.c - Two threads increment and decrement a shared counter without synchronization.</b>",
    h="100%", 
    w="100%"
) + '</font>'))

<font size="1.2rem"><b>counter.c - Two threads increment and decrement a shared counter without synchronization.</b>
<div style="width:100%; height:100%; font-size:inherit; overflow: auto;" >


``` 
0: #include <pthread.h>
1: #include <stdlib.h>
2: #include <sys/types.h>
3: #include <unistd.h>
4: #include <string.h>
5: #include <stdio.h>
6: 
7: pthread_mutex_t mutex = PTHREAD_MUTEX_INITIALIZER;
8: volatile long shared_counter;
9: 
10: static void *increment_thread(void *arg)
11: {
12:     long niters = (long)arg;
13:     for (int i = 0; i < niters; i++) {
14:         pthread_mutex_lock(&mutex);
15:         shared_counter++;
16:         pthread_mutex_unlock(&mutex);
17:     }
18: 
19:     return (void *)0;
20: }
21: 
22: static void *decrement_thread(void *arg)
23: {
24:     long niters = (long)arg;
25: 
26:     for (int i = 0; i < niters; i++) {
27:         pthread_mutex_lock(&mutex);
28:         shared_counter--;
29:         pthread_mutex_unlock(&mutex);
30:     }
31: 
32:     return (void *)0;
33: }
34: 
35: int main(int argc, char **argv)
36: {
37:     pthread_t *tids;
38:     long niters;
39:     int nthreads, i;
40:     
41:     if (argc != 3) {
42:         fprintf(stderr,"Usage: %s <num_threads> <num_iters>\n",argv[0]);
43:         return 1;
44:     }
45: 
46:     nthreads = atoi(argv[1]);
47:     niters = atoi(argv[2]);
48:     shared_counter = 0;
49:     
50:     printf("Main thread: Beginning test with %d threads\n", nthreads);
51:     
52:     tids = (pthread_t *)malloc(nthreads * sizeof(pthread_t));
53:     
54:     /* We create the same number of increment and decrement threads, each doing the same number of iterations. 
55:      * When all threads have completed, we expect the final value of the shared counter to be the same as its
56:      * initial value (i.e., 0).
57:      */
58:     for (i = 0; i < nthreads; i+=2) {
59:         (void)pthread_create(&tids[i], NULL, increment_thread, (void *)niters );
60:         (void)pthread_create(&tids[i+1], NULL, decrement_thread, (void *)niters );
61:     }
62:     
63:     /* Wait for child threads to finish */
64:     for (i = 0; i < nthreads; i+=2) {
65:         pthread_join(tids[i], NULL);
66:         pthread_join(tids[i+1], NULL);
67:     }
68:     
69:     printf("Main thread: Final value of shared counter is %ld\n", shared_counter);
70: 
71:     return 0;
72: }

```


</div>
</font>

In [4]:
bash.runNoOutput("[[ -a counter_mutex ]] && rm counter_mutex;make counter_mutex")
cmds = '''./counter_mutex 2 20000
./counter_mutex 2 100000
./counter_mutex 4 100000
./counter_mutex 4 100000'''

bash.run(cmds)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…